In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def compute_polys(dx, dy):
    n = len(dx)
    np1 = n + 1
    
    # Step 1
    a = np.empty(np1)
    a[0] = 3 * dy[0] / dx[0]
    a[1:-1] = 3 * dy[1:] / dx[1:] - 3 * dy[:-1] / dx[:-1]
    a[-1] = -3 * dy[-1] / dx[-1]
    
    # Step 2
    l = np.empty(np1)
    u = np.empty(n)
    z = np.empty(np1)
    l[0] = 2*dx[0]
    u[0] = 0.5
    z[0] = a[0] / l[0]
    for i in range(1, n):
        l[i] = 2*dx[i] + dx[i-1] * (2 - u[i-1])
        u[i] = dx[i] / l[i]
        z[i] = (a[i] - dx[i-1] * z[i-1]) / l[i]
    l[-1] = dx[-1] * (2 - u[-1])
    z[-1] = (a[-1] - dx[-1] * z[-2]) / l[-1]
    
    # Step 3
    c = np.empty(np1)
    c[-1] = z[-1]
    for j in range(n-1, -1, -1):
        c[j] = z[j] - u[j] * c[j+1]
        
    # Step 4
    b = dy / dx - dx * (c[1:] + 2*c[:-1]) / 3
    d = (c[1:] - c[:-1]) / (3*dx)
    
    return (np.vstack((
        np.concatenate(([0.0], b, [0.0])),
        np.concatenate(([0.0], c[:-1], [0.0])),
        np.concatenate(([0.0], d, [0.0]))
    )).T, a, z, u, l)

def compute_polys_rev(dx, dy, P, a, z, u, l, bP):
    n = len(dx)
    np1 = n + 1

    b = P[1:-1, 0]
    c = P[1:, 1]
    d = P[1:-1, 2]
    
    bb = np.array(bP[1:-1, 0])
    bc = np.array(bP[1:, 1])
    bd = np.array(bP[1:-1, 2])
    bc[-1] = 0.0
    
    # Step 4
    # d = (c[1:] - c[:-1]) / (3*dx)
    bdx = -d * bd / dx
    bc[1:] += bd / (3*dx)
    bc[:-1] += -bd / (3*dx)
    
    # b = dy / dx - dx * (c[1:] + 2*c[:-1]) / 3
    bdy = bb / dx
    bdx += -(dy/dx**2 + (c[1:]+2*c[:-1])/3) * bb
    bc[1:] += -dx * bb / 3
    bc[:-1] += -2 * dx * bb / 3
    
    # Step 3
    bu = np.zeros_like(u)
    bz = np.zeros_like(z)
    for j in range(n):
        # c[j] = z[j] - u[j] * c[j+1]
        bz[j] += bc[j]
        bc[j+1] += -bc[j] * u[j]
        bu[j] += -c[j+1] * bc[j]
    # c[-1] = z[-1]
    bz[-1] += bc[-1]
    
    # Step 2
    # z[-1] = (a[-1] - dx[-1] * z[-2]) / l[-1]
    ba = np.zeros_like(a)
    bl = np.zeros_like(l)
    ba[-1] += bz[-1] / l[-1]
    bdx[-1] += -z[-2] * bz[-1] / l[-1]
    bz[-2] += -dx[-1] * bz[-1] / l[-1]
    bl[-1] += -z[-1] * bz[-1] / l[-1]
    
    # l[-1] = dx[-1] * (2 - u[-1])
    bdx[-1] += (2 - u[-1]) * bl[-1]
    bu[-1] += -dx[-1] * bl[-1]

    for i in range(n-1, 0, -1):
        # z[i] = (a[i] - dx[i-1] * z[i-1]) / l[i]
        ba[i] += bz[i] / l[i]
        bl[i] += -z[i]*bz[i]/l[i]
        bdx[i-1] += -z[i-1] * bz[i] / l[i]
        bz[i-1] += -bz[i] * dx[i-1] / l[i]
        
        # u[i] = dx[i] / l[i]
        bdx[i] += bu[i] / l[i]
        bl[i] += -bu[i]*u[i]/l[i]

        # l[i] = 2*dx[i] + dx[i-1] * (2 - u[i-1])
        bdx[i] += 2*bl[i]
        bdx[i-1] += (2-u[i-1])*bl[i]
        bu[i-1] += -dx[i-1] * bl[i]
        
    # z[0] = a[0] / l[0]
    ba[0] += bz[0] / l[0]
    bl[0] += -z[0] * bz[0] / l[0]

    # l[0] = 2*dx[0]
    bdx[0] += 2*bl[0]
    
    # Step 1
    # a[0] = 3 * dy[0] / dx[0]
    bdy[0] += 3 * ba[0] / dx[0]
    bdx[0] += -a[0] * ba[0] / dx[0]

    # a[1:-1] = 3 * dy[1:] / dx[1:] - 3 * dy[:-1] / dx[:-1]
    bdy[1:] += 3 * ba[1:-1] / dx[1:]
    bdy[:-1] += -3 * ba[1:-1] / dx[:-1]
    bdx[1:] += -3 * dy[1:] * ba[1:-1] / dx[1:]**2
    bdx[:-1] += 3 * dy[:-1] * ba[1:-1] / dx[:-1]**2

    # a[-1] = -3 * dy[-1] / dx[-1]
    bdy[-1] += -3 * ba[-1] / dx[-1]
    bdx[-1] += -a[-1] * ba[-1] / dx[-1]
    
    return bdx, bdy

In [ ]:
np.random.seed(42)
x = np.sort(np.random.uniform(1, 9, 61))
# x = np.linspace(1, 9, 61)
y = np.sin(x)
dx = np.diff(x)
dy = np.diff(y)

P, a, z, u, l = compute_polys(dx, dy)
bP = np.zeros_like(P)
# inds = ([3, 3, 3], [0, 1, 2])
inds = tuple(a.flatten() for a in np.indices(bP.shape))
bP[inds] = 1.0
# print(bP)
bx, by = compute_polys_rev(dx, dy, P, a, z, u, l, bP)
print(bx)
print(by)

In [ ]:
value = dx
grad = bx

eps = 1e-8

for i in range(len(value)):
    value[i] += eps
    r = compute_polys(dx, dy)
    vp = np.sum(r[0][inds])

    value[i] -= 2*eps
    r = compute_polys(dx, dy)
    vm = np.sum(r[0][inds])
    value[i] += eps

    est = 0.5 * (vp - vm) / eps
    print(est, grad[i], est - grad[i], (est - grad[i]) / grad[i])

In [ ]:
t = np.linspace(0, 10, 500)
inds = np.searchsorted(x, t)

xp = np.concatenate((x[:1], x, x[-1:]))
yp = np.concatenate((y[:1], y, y[-1:]))

poly = P[inds]
dd = t - xp[inds]
value = yp[inds] + poly[:, 0] * dd + poly[:, 1] * dd**2 + poly[:, 2] * dd**3

plt.plot(t, np.sin(t))
plt.plot(t, value)
plt.plot(x, y, ".")